In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

def stock_code(option):
    if option == 'kospi':
        df = pd.read_html('../database/stock_code/raw/stock_code_kospi_raw.xls')[0]
    elif option == 'kosdak':
        df = pd.read_html('../database/stock_code/raw/stock_code_kosdak_raw.xls')[0]
    else:
        print('No such table')
    df = df.rename(columns = dict(회사명='StockName', 종목코드='StockCode', 업종='TypeBiz', 주요제품='MainProduct',상장일='OriginDate'))
    df = df[['StockName','StockCode','TypeBiz','MainProduct','OriginDate']]
    return df

def explore_crawling(stock_code):
    url = 'https://finance.naver.com/item/sise_day.nhn?code=' + str(stock_code).zfill(6) + '&page=1'
    temp = pd.read_html(url)[0].dropna(axis=0)
    if len(temp) == 0:
        output = np.NaN
    else:
        output = temp['날짜'].iloc[0][:4]
    return output
    
def job_collect(option):
    df = stock_code(option)
    df['LastYear'] = df['StockCode'].apply(lambda x: explore_crawling(x))
    df = df[df['LastYear']=='2020'].reset_index(drop=True).drop(columns='LastYear').copy()
    df.to_csv(f'../database/stock_code/stock_code_{option}.csv', index=False)
    df.to_csv(f'../database/stock_code/stock_code_{option}_{today}.csv', index=False)

In [ ]:
job_collect(option = 'kospi')